In [ ]:
!pip install lightkurve astroquery astropy numpy pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 134.5 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=b95f4ee2b1672661238353b62d48f04698e26b64c01f3573b8ff39212111c6aa
  Stored in directory: /root/.cache/pip/wheels/04/15/cd/2f622795b09e83471a3be5d2581cd9cf96a6ec7aa78e8deffe
  Created wheel for memoization

In [ ]:
# exoplanet_classifier.py
# Python 3.10+
# Requirements:
#   pip install astroquery astropy pandas scikit-learn numpy

import os
import numpy as np
import pandas as pd

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import f1_score, balanced_accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.utils.class_weight import compute_sample_weight


# --------------------------
# 1) UTILITAIRES DOWNLOAD
# --------------------------

def df_from_table(table):
    df = table.to_pandas()
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[()]+", "", regex=True)
    )
    return df


def fetch_koi():
    t = NasaExoplanetArchive.query_criteria(
        table="koi",
        select="kepid, kepoi_name, koi_disposition, koi_period, koi_duration, koi_depth, koi_model_snr, "
               "koi_steff, koi_slogg, koi_srad, koi_kepmag, koi_fpflag_nt, koi_fpflag_ss, koi_fpflag_co, koi_fpflag_ec",
        cache=True
    )
    df = df_from_table(t)
    df["mission"] = "KEPLER"
    return df


def fetch_k2():
    """
    K2 Planets and Candidates (table = 'k2pandc').
    Colonnes PS: pl_orbper (jours), pl_trandep (ppm), pl_trandur (jours), st_*, sy_*.
    """
    from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

    t = NasaExoplanetArchive.query_criteria(
        table="k2pandc",
        select=(
            "epic_hostname, k2_name, hostname, disposition, "
            "pl_orbper, pl_trandep, pl_trandur, "
            "st_teff, st_logg, st_rad, "
            "sy_kepmag, sy_tmag"
        ),
        cache=True
    )
    df = t.to_pandas()
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[()]+", "", regex=True)
    )

    df.rename(columns={
        "epic_hostname": "epic_hostname",
        "disposition": "Archive_Disposition",
        "pl_orbper": "period",
        "pl_trandur": "duration_d",
        "pl_trandep": "depth",
        "st_teff": "st_teff",
        "st_logg": "st_logg",
        "st_rad": "st_rad",
        "sy_kepmag": "kepmag",
        "sy_tmag": "tmag",
    }, inplace=True)

    df["object_id"] = (
        df.get("epic_hostname")
          .fillna(df.get("k2_name"))
          .fillna(df.get("hostname"))
          .astype(str)
    )

    df["star_id"] = df.get("epic_hostname").fillna(df.get("hostname")).astype(str)

    # magnitude unique
    df["mag"] = df.get("kepmag").fillna(df.get("tmag"))

    df["duration_h"] = df["duration_d"] * 24.0

    df["mission"] = "K2"
    return df

def fetch_toi():
    """
    TESS Objects of Interest (table = 'toi').
    Colonnes clefs (PS schema):
      - tid (TIC ID), tfopwg_disp (disposition),
      - pl_orbper [days], pl_trandurh [hours], pl_trandep [ppm],
      - st_teff, st_logg, st_rad, st_tmag (TESS magnitude).
    Doc colonnes: https://exoplanetarchive.ipac.caltech.edu/docs/API_TOI_columns.html
    """
    from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

    t = NasaExoplanetArchive.query_criteria(
        table="toi",
        select=(
            "toi, tid, tfopwg_disp, "
            "pl_orbper, pl_trandurh, pl_trandep, "
            "st_teff, st_logg, st_rad, st_tmag"
        ),
        cache=True
    )
    df = t.to_pandas()
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[()]+", "", regex=True)
    )

    # Renommage vers notre schéma commun
    df.rename(columns={
        "tid": "TIC_ID",
        "tfopwg_disp": "TFOPWG_Disposition",
        "pl_orbper": "Period_days",
        "pl_trandurh": "Duration_hours",
        "pl_trandep": "Depth_ppm",
        "st_tmag": "TESS_Mag"
    }, inplace=True)

    df["mission"] = "TESS"
    return df


# --------------------------
# 2) HARMONISATION
# --------------------------

def harmonize_koi(df):
    return pd.DataFrame({
        "object_id": df.get("kepoi_name", df.index.astype(str)),
        "mission": "KEPLER",
        "star_id": df.get("kepid"),
        "period": df.get("koi_period"),
        "duration": df.get("koi_duration"),
        "depth": df.get("koi_depth"),
        "snr": df.get("koi_model_snr"),
        "st_teff": df.get("koi_steff"),
        "st_logg": df.get("koi_slogg"),
        "st_rad": df.get("koi_srad"),
        "mag": df.get("koi_kepmag"),
        "fpflag_nt": df.get("koi_fpflag_nt"),
        "fpflag_ss": df.get("koi_fpflag_ss"),
        "fpflag_co": df.get("koi_fpflag_co"),
        "fpflag_ec": df.get("koi_fpflag_ec"),
        "label_raw": df.get("koi_disposition"),
    })


def harmonize_k2(df):
    return pd.DataFrame({
        "object_id": df.get("object_id").astype(str),
        "mission": "K2",
        "star_id": df.get("star_id").astype(str),
        "period": df.get("period"),
        "duration": df.get("duration_h"),
        "depth": df.get("depth"),
        "snr": df.get("snr"),
        "st_teff": df.get("st_teff"),
        "st_logg": df.get("st_logg"),
        "st_rad": df.get("st_rad"),
        "mag": df.get("mag"),
        "fpflag_nt": df.get("fpflag_nt"),
        "fpflag_ss": df.get("fpflag_ss"),
        "fpflag_co": df.get("fpflag_co"),
        "fpflag_ec": df.get("fpflag_ec"),
        "label_raw": df.get("Archive_Disposition"),
    })



def harmonize_toi(df):
    def map_toi_label(x):
        if pd.isna(x):
            return None
        x = str(x).upper()
        if x in {"KP", "CP", "CONFIRMED"}:
            return "CONFIRMED"
        if x in {"PC", "APC", "CANDIDATE"}:
            return "CANDIDATE"
        if x in {"FP", "FALSE POSITIVE"}:
            return "FALSE POSITIVE"
        return None

    return pd.DataFrame({
        "object_id": df.get("toi").astype(str),
        "mission": "TESS",
        "star_id": df.get("TIC_ID"),
        "period": df.get("Period_days"),
        "duration": df.get("Duration_hours"),
        "depth": df.get("Depth_ppm"),
        "snr": df.get("SNR"),
        "st_teff": df.get("st_teff"),
        "st_logg": df.get("st_logg"),
        "st_rad": df.get("st_rad"),
        "mag": df.get("TESS_Mag"),
        "fpflag_nt": df.get("fpflag_nt"),
        "fpflag_ss": df.get("fpflag_ss"),
        "fpflag_co": df.get("fpflag_co"),
        "fpflag_ec": df.get("fpflag_ec"),
        "label_raw": df.get("TFOPWG_Disposition").map(map_toi_label),
    })


# --------------------------
# 3) PIPELINE ML
# --------------------------

def run_classifier(harm):
    from sklearn.preprocessing import QuantileTransformer, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    from sklearn.ensemble import HistGradientBoostingClassifier
    from sklearn.metrics import f1_score, balanced_accuracy_score
    from sklearn.model_selection import StratifiedGroupKFold
    from sklearn.utils.class_weight import compute_sample_weight

    label_map = {"CONFIRMED": 2, "CANDIDATE": 1, "FALSE POSITIVE": 0}
    harm = harm[harm["label_raw"].isin(label_map.keys())].copy()
    harm["label"] = harm["label_raw"].map(label_map).astype(int)

    for c in ["period", "duration", "depth"]:
        if c in harm.columns:
            harm.loc[(~harm[c].isna()) & (harm[c].astype(float) <= 0), c] = np.nan

    base_num_cols = [
        "period", "duration", "depth", "snr",
        "st_teff", "st_logg", "st_rad", "mag",
        "fpflag_nt", "fpflag_ss", "fpflag_co", "fpflag_ec",
    ]

    eps = 1e-6
    harm["log_period"]          = np.log10(harm["period"].astype(float) + eps)
    harm["log_duration_h"]      = np.log10(harm["duration"].astype(float) + eps)
    harm["log_depth_ppm"]       = np.log10(harm["depth"].astype(float) + eps)
    harm["depth_over_duration"] = harm["depth"].astype(float) / (harm["duration"].astype(float) + eps)

    derived_cols = ["log_period", "log_duration_h", "log_depth_ppm", "depth_over_duration"]
    base_num_cols = [c for c in base_num_cols if c in harm.columns]
    all_num_cols = base_num_cols + derived_cols

    cat_cols = ["mission"]

    y = harm["label"].values
    groups_star = harm["star_id"].astype(str).values
    groups_mission = harm["mission"].values

    clf = HistGradientBoostingClassifier(
        learning_rate=0.08,
        max_iter=500,
        max_leaf_nodes=31,
        early_stopping=True,
        random_state=42
    )

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    f1s, bals = [], []

    for fold, (tr, te) in enumerate(cv.split(harm, y, groups_star), 1):
        df_tr = harm.iloc[tr].copy()
        df_te = harm.iloc[te].copy()

        num_cols_fit = [c for c in all_num_cols if df_tr[c].notna().any()]
        pre = ColumnTransformer([
            ("num", Pipeline([
                ("imp", SimpleImputer(strategy="median")),
                ("qt",  QuantileTransformer(output_distribution="normal",
                                            subsample=200_000, random_state=42)),
            ]), num_cols_fit),
            ("cat", Pipeline([
                ("imp", SimpleImputer(strategy="most_frequent")),
                ("oh", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols),
        ])

        pipe = Pipeline([("pre", pre), ("clf", clf)])
        sw = compute_sample_weight(class_weight="balanced", y=y[tr])
        pipe.fit(df_tr[num_cols_fit + cat_cols], y[tr], clf__sample_weight=sw)
        y_hat = pipe.predict(df_te[num_cols_fit + cat_cols])

        f1s.append(f1_score(y[te], y_hat, average="macro"))
        bals.append(balanced_accuracy_score(y[te], y_hat))
        print(f"[Fold {fold}] F1-macro={f1s[-1]:.3f}, BalAcc={bals[-1]:.3f}")

    print(f"\nCV results: F1-macro={np.mean(f1s):.3f}±{np.std(f1s):.3f}, "
          f"BalAcc={np.mean(bals):.3f}±{np.std(bals):.3f}")

    # ---------- Leave-One-Mission-Out () ----------
    print("\nLeave-One-Mission-Out:")
    for m in np.unique(groups_mission):
        tr_idx = np.where(groups_mission != m)[0]
        te_idx = np.where(groups_mission == m)[0]
        if len(te_idx) < 50:
            continue

        df_tr = harm.iloc[tr_idx].copy()
        df_te = harm.iloc[te_idx].copy()

        num_cols_fit = [c for c in all_num_cols if df_tr[c].notna().any()]

        pre = ColumnTransformer([
            ("num", Pipeline([
                ("imp", SimpleImputer(strategy="median")),
                ("qt",  QuantileTransformer(output_distribution="normal",
                                            subsample=200_000, random_state=42)),
            ]), num_cols_fit),
        ])

        pipe = Pipeline([("pre", pre), ("clf", clf)])
        sw = compute_sample_weight(class_weight="balanced", y=y[tr_idx])

        pipe.fit(df_tr[num_cols_fit], y[tr_idx], clf__sample_weight=sw)
        y_hat = pipe.predict(df_te[num_cols_fit])

        f1 = f1_score(y[te_idx], y_hat, average="macro")
        bal = balanced_accuracy_score(y[te_idx], y_hat)
        print(f"Train≠{m} → Test={m}: F1-macro={f1:.3f}, BalAcc={bal:.3f}")


# --------------------------
# 4) MAIN
# --------------------------

if __name__ == "__main__":
    os.makedirs("data", exist_ok=True)

    print("Fetching NASA catalogs via astroquery...")
    koi_df = fetch_koi()
    k2_df = fetch_k2()
    toi_df = fetch_toi()

    koi_df.to_csv("data/cumulative_koi.csv", index=False)
    k2_df.to_csv("data/k2planets.csv", index=False)
    toi_df.to_csv("data/toi.csv", index=False)

    print("Harmonizing...")
    harm = pd.concat([
        harmonize_koi(koi_df),
        harmonize_k2(k2_df),
        harmonize_toi(toi_df)
    ], ignore_index=True)

    num_cols_all = harm.select_dtypes(include=[np.number]).columns
    nan_ratio = harm[num_cols_all].isna().mean()
    cols_to_drop = nan_ratio[nan_ratio > 0.95].index.tolist()
    if cols_to_drop:
        print(f"[Info] Dropping quasi-empty numeric columns (>95% NaN): {cols_to_drop}")
        harm.drop(columns=cols_to_drop, inplace=True)

    harm.to_csv("data/exoplanets_harmonized.csv", index=False)

    print(f"Harmonized dataset shape: {harm.shape}")
    print(harm["label_raw"].value_counts())

    print("\nRunning classifier...")
    run_classifier(harm)

Fetching NASA catalogs via astroquery...
Harmonizing...


/tmp/ipython-input-3920245596.py:373: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  harm = pd.concat([


Harmonized dataset shape: (66358, 16)
label_raw
CONFIRMED            24203
FALSE POSITIVE       22655
CANDIDATE            15308
NOT DISPOSITIONED     4072
REFUTED                 22
Name: count, dtype: int64

Running classifier...
[Fold 1] F1-macro=0.830, BalAcc=0.832
[Fold 2] F1-macro=0.829, BalAcc=0.831
[Fold 3] F1-macro=0.835, BalAcc=0.840
[Fold 4] F1-macro=0.831, BalAcc=0.834
[Fold 5] F1-macro=0.830, BalAcc=0.832

CV results: F1-macro=0.831±0.002, BalAcc=0.834±0.003

Leave-One-Mission-Out:
Train≠K2 → Test=K2: F1-macro=0.254, BalAcc=0.357
Train≠KEPLER → Test=KEPLER: F1-macro=0.346, BalAcc=0.398
Train≠TESS → Test=TESS: F1-macro=0.384, BalAcc=0.402


Intra-mission (CV par étoile) :
Le modèle fonctionne très bien → F1-macro ≈ 0.83, Balanced Accuracy ≈ 0.83.
👉 Interprétation : bonne capacité de généralisation à l’intérieur d’une même mission (Kepler, K2, TESS).

Cross-mission (LOMO) :
Quand on entraîne sur 2 missions et qu’on teste sur la 3ᵉ, les performances chutent fortement (F1-macro ~0.25–0.38).
👉 Interprétation : le modèle apprend surtout des indices spécifiques aux missions (bruit instrumental, magnitudes, snr, critères de catalogage) → il ne généralise pas vers un nouveau survey.

Probleme :

Si l’objectif est de classifier dans une mission donnée → pas de problème, on peut ignorer le LOMO.

Si l’objectif est de construire un modèle universel inter-mission → il faut travailler sur des features physiques invariantes (ratios, normalisations par étoile, paramètres orbitaux/stellaires dérivés) et/ou envisager des techniques d’adaptation de domaine.